In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def normalize(column):
    """ Normalize the column to a range between 0 and 1 """
    return (column.max() - column) / (column.max() - column.min())

def convert_importance(importance):
    """ Convert categorical importance to numerical values """
    mapping = {
        "Very Important": 3,
        "Neutral": 2,
        "Less Importance": 1
    }
    return importance.map(mapping)

def calculate_remaining_time(deadline_dates, deadline_times):
    """ Calculate remaining time from now until the deadline """
    current_datetime = datetime.now()
    deadline_datetimes = pd.to_datetime(deadline_dates + ' ' + deadline_times, format='%Y-%m-%d %I:%M %p')
    return (deadline_datetimes - current_datetime).dt.total_seconds() / 3600  # Convert seconds to hours

def calculate_daily_plan(df, days=7):
    """ Calculate the daily plan for each task over the specified number of days """
    # Calculate remaining time for the deadlines
    df['Remaining Time'] = calculate_remaining_time(df['Expected Deadline'], df['Actual Time to Submit'])
    
    # Normalize the columns
    df['ED_norm'] = normalize(df['Remaining Time'])
    df['ETC_norm'] = normalize(df['Expected Time to Complete'])
    df['RT_norm'] = df['Recurring Task'].apply(lambda x: 0 if x.lower() == 'yes' else 1)
    df['DI_norm'] = normalize(convert_importance(df['Degree of Importance']))
    
    # Calculate the total weight for each task
    df['Total Weight'] = df['ED_norm'] + df['ETC_norm'] + df['RT_norm'] + df['DI_norm']
    
    # Calculate the daily plan
    daily_plan = {}
    current_date = datetime.now()
    for i in range(days):
        daily_plan[current_date.strftime('%Y-%m-%d')] = {}
        total_available_time = 24  # Hours in a day
        for index, row in df.iterrows():
            work_name = row['Work Name']
            weight = row['Total Weight']
            proportion_time = weight / df['Total Weight'].sum()
            daily_time = total_available_time * proportion_time
            daily_plan[current_date.strftime('%Y-%m-%d')][work_name] = daily_time
        current_date += timedelta(days=1)  # Move to the next day
    return daily_plan

def main():
    # Load the CSV file
    file_path = input("Enter the path to the CSV file: ")
    df = pd.read_csv(file_path)

    # Calculate the daily plan for the tasks over the next 7 days
    daily_plan = calculate_daily_plan(df)

    # Output the daily plan
    print("Daily Plan for the next 7 days:")
    for date, tasks in daily_plan.items():
        print(f"On {date}:")
        for task, time in tasks.items():
            print(f"- {task}: {time:.2f} hours")

if __name__ == "__main__":
    main()
